In [1]:
'''The code aims to classify the handwritten words, which is classification in this case.'''
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

category=12
import numpy as np

In [2]:
def load_data():
    x=[]
    y=[]
    x_test=[]
    y_test=[]
    
    for i in range(category):
        root = f'../train/{i+1}'
        filenames = os.listdir(root)
        np.random.shuffle(filenames)  
        test_num = 0
        for filename in filenames:
            img_root = os.path.join(root, filename)
            image = Image.open(img_root)
            if test_num < (int)(0.2*len(filenames)):
                x_test.append(np.array(image).flatten())
                y_test.append([1 if j == i else 0 for j in range(12)])
                test_num += 1
            else:
                x.append(np.array(image).flatten())
                y.append([1 if j == i else 0 for j in range(12)])
    
    x = np.array(x)
    y = np.array(y)
    x_test = np.array(x_test)
    y_test = np.array(y_test)
    
    return x, y, x_test, y_test

In [83]:
def make_MLP(input, output):
    W1 = np.random.randn(input, output)/np.sqrt(input*output)
    b1 = np.random.randn(output)/np.sqrt(output)
    return W1, b1
def sigmoid(x):
    return 1/(1+np.exp(-x))

class Layer():
    def __init__(self, input, output,activation=""):
        self.W1, self.b1= make_MLP(input, output)
        self.activation = activation
    def forward(self,x):
        self.x=x
        self.z1 = np.dot(x, self.W1) + self.b1
        self.a1= self.z1
        if self.activation=="sigmoid":
            self.a1 = sigmoid(self.z1)
        return self.a1
    def backward(self,lossdx,learning_rate):
        # x y is the batch data
        dW1 = np.zeros_like(self.W1)  # (input, hidden_layer)
        db1 = np.zeros_like(self.b1)  # (hidden_layer)

        z1 = self.z1
        a1 = self.a1 
        
        if self.activation=="sigmoid":
            lossdx =  a1 * (1-a1) *lossdx
        print("the shape of lossdx",lossdx.shape)
        print("the shape of x",self.x.shape)
        print("the shape of W1",self.W1.shape)
        print("the shape of dW1",dW1.shape)
        print("the shape of db1",db1.shape)
        print("the shape of z1",z1.shape)
        print("the shape of a1",a1.shape)
        db1 += lossdx
        try:
            dW1 += np.dot(self.x,lossdx.T)
        except:
            self.x=self.x.reshape(-1,1)
            lossdx=lossdx.reshape(-1,1)
            dW1 += np.dot(self.x,lossdx.T)
        
        lossdx = np.dot(self.W1,lossdx)
        self.W1 -= learning_rate*dW1/len(self.x) # /the batch size
        self.b1 -= learning_rate*db1/len(self.x)
        print("the shape of lossdx",lossdx.shape)
        print("************************")
        return lossdx.squeeze()

In [84]:
a=np.array([1,2,3,4,5,6,7])
net=Layer(7,5,"sigmoid")
print(net.forward(a))

[0.68526785 0.19487044 0.69000968 0.21042554 0.00480183]


In [85]:
net.backward(np.array([1,1,1,1,1]),0.1)

the shape of lossdx (5,)
the shape of x (7,)
the shape of W1 (7, 5)
the shape of dW1 (7, 5)
the shape of db1 (5,)
the shape of z1 (5,)
the shape of a1 (5,)
the shape of lossdx (7, 1)
************************


array([-0.09262614,  0.10439036, -0.16193458,  0.01597333, -0.08037689,
       -0.0568763 ,  0.09800388])

In [86]:
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x))

In [87]:
class MLP():
    def __init__(self):
        self.layers=[]
    def add_layer(self,layer):
        self.layers.append(layer)
    def forward(self,x):
        for layer in self.layers:
            x=layer.forward(x)
        self.output=x
        x=softmax(x)
        return x
    def backward(self,y,learning_rate):
        # we first calculate the lossdx of the last layer
        lossdx = y-self.output
        for layer in self.layers[::-1]:
            lossdx=layer.backward(lossdx,learning_rate)
    
        

In [88]:
mlp=MLP()
mlp.add_layer(Layer(5,9,"sigmoid"))
mlp.add_layer(Layer(9,3))
mlp.forward(np.array([1,2,3,4,5]))

array([0.2441439 , 0.24902651, 0.5068296 ])

In [89]:
mlp.backward(np.array([1,0,0]),0.1)

the shape of lossdx (3,)
the shape of x (9,)
the shape of W1 (9, 3)
the shape of dW1 (9, 3)
the shape of db1 (3,)
the shape of z1 (3,)
the shape of a1 (3,)
the shape of lossdx (9, 1)
************************
the shape of lossdx (9,)
the shape of x (5,)
the shape of W1 (5, 9)
the shape of dW1 (5, 9)
the shape of db1 (9,)
the shape of z1 (9,)
the shape of a1 (9,)
the shape of lossdx (5, 1)
************************
